Divide periods into individual files

In [1]:
import pandas as pd

# Loading CSV file
#df = pd.read_csv('F:\\data_all\\CollegeMsg\\data.csv')
df = pd.read_csv('G:\\Bluetooth\\three_period\\bt_symmetric.csv')

# Converts the Unix timestamp for the 'time' column to date-time format and adds it as a new column
# Assuming the timestamp is in seconds, if it is in milliseconds, set unit='ms'
#df['datetime'] = pd.to_datetime(df['time'], unit='s')
df['datetime'] = pd.to_datetime(df['# timestamp'], unit='s')

# Determine the minimum and maximum values of the converted date time
min_time = df['datetime'].min()
max_time = df['datetime'].max()

# Calculate the total time range and divide by 6 to get the length of each time period
time_interval = (max_time - min_time) / 3

# Create and save a file for each time period
for i in range(3):
    start_time = min_time + i * time_interval
    end_time = start_time + time_interval
    
    # Filter data by time interval
    interval_df = df[(df['datetime'] >= start_time) & (df['datetime'] < end_time)]
    
    # Save to a new CSV file
    interval_df.to_csv(f'G:\\Bluetooth\\three_period\\data_{i+1}.csv', index=False)

# This will create 6 files: data_1.csv to data_6.csv,
# Each file contains data for the corresponding time period, keeping the original Unix timestamp in the third column and the converted date-time in the new fourth column.

Construct the number of calls feature

In [22]:
import pandas as pd
import os

file_names = [f'G:\\data_all\\askubuntu_c2q\\N\\data_{i}.csv' for i in range(1, 7)]
#file_names = [f'G:\\Bluetooth\\three_period\\data_{i}.csv' for i in range(1, 4)]

for file_name in file_names:

    df = pd.read_csv(file_name)
    
    interaction_counts = df.groupby(['ego', 'alter']).size().reset_index(name='weight')
    #interaction_counts = df.groupby(['user_a', 'user_b']).size().reset_index(name='weight')
    
    dir_name, base_name = os.path.split(file_name)
    
    output_file_name = os.path.join(dir_name, f'N_{base_name}')
    
    interaction_counts.to_csv(output_file_name, index=False)

It is present in all periods, and the number of users and their contacts in each period is greater than or equal to 10

In [4]:
import pandas as pd

#file_names = [f'F:\\data_all\\CollegeMsg\\N_data_{i}.csv' for i in range(1, 4)]
file_names = [f'G:\\data_all\\askubuntu_c2q\\N\\data_{i}.csv' for i in range(1, 7)]
results = []

for i, file_name in enumerate(file_names, start=1):
    df = pd.read_csv(file_name)
    counts = df.groupby('ego')['alter'].nunique().reset_index(name=f'alters_period_{i}')
    counts.set_index('ego', inplace=True)
    results.append(counts)

combined_df = pd.concat(results, axis=1)

filtered_df = combined_df.dropna()

filtered_df = filtered_df[filtered_df.apply(lambda x: all(x >= 10), axis=1)]

filtered_df.to_csv('G:\\data_all\\askubuntu_c2q\\filtered_results.csv')

Reciprocity of calls

In [33]:
import pandas as pd

#file_names = [f'F:\\data_all\\askubuntu_a2q\\N\\N_data_{i}.csv' for i in range(1, 7)]
file_names = [f'G:\\data_all\\askubuntu_c2q\\N\\data_{i}.csv' for i in range(1, 7)]

for file_index, file_name in enumerate(file_names, start=1):

    df = pd.read_csv(file_name)

    r_values = []

    for index, row in df.iterrows():
        i = row['ego']
        j = row['alter']
        weight = row['weight']
        
        i_to_j_interactions = df[(df['ego'] == i) & (df['alter'] == j)]['weight'].sum()
        
        j_to_i_interactions = df[(df['ego'] == j) & (df['alter'] == i)]['weight'].sum()
        
        total_interactions = i_to_j_interactions + j_to_i_interactions
        
        r = abs((i_to_j_interactions / total_interactions) - 0.5)
        
        r_values.append(r)

    df['weight'] = r_values

    output_file_name = f'G:\\data_all\\askubuntu_c2q\\R\\R_data_{file_index}.csv'

    df.to_csv(output_file_name, index=False)

Topological overlap

In [24]:
import pandas as pd

for i in range(1, 7):
    #input_file_path = f'F:\\data_all\\askubuntu_a2q\\N\\N_data_{i}.csv'  # 注意文件扩展名是.csv
    #output_file_path = f'F:\\data_all\\askubuntu_a2q\\O\\O_data_{i}.csv'
    input_file_path = f'G:\\data_all\\askubuntu_c2q\\N_data_{i}.csv'  # 注意文件扩展名是.csv
    output_file_path = f'G:\\data_all\\askubuntu_c2q\\O\\O_data_{i}.csv'
    
    df = pd.read_csv(input_file_path)
    
    o_values = []
    for index, row in df.iterrows():
        i_neighbors_a = set(df[df['ego'] == row['ego']]['weight'])
        i_neighbors_b = set(df[df['alter'] == row['ego']]['weight'])
        i_neighbors = i_neighbors_a.union(i_neighbors_b)

        j_neighbors_a = set(df[df['ego'] == row['alter']]['weight'])
        j_neighbors_b = set(df[df['alter'] == row['alter']]['weight'])
        j_neighbors = j_neighbors_a.union(j_neighbors_b)

        intersection = len(i_neighbors.intersection(j_neighbors))
        union = len(i_neighbors.union(j_neighbors))
        
        o_value = intersection / union if union != 0 else 0
        o_values.append(o_value)

    df['weight'] = o_values

    df.to_csv(output_file_path, index=False)
    
    print(f'File {i} processed and saved to {output_file_path}')

File 1 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_1.csv
File 2 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_2.csv
File 3 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_3.csv
File 4 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_4.csv
File 5 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_5.csv
File 6 processed and saved to G:\data_all\askubuntu_c2q\O\O_data_6.csv


Connectivity diversity

In [25]:
import pandas as pd
import math

for i in range(1, 7):
    #input_file_path = f'F:\\data_all\\askubuntu_a2q\\N\\N_data_{i}.csv'  # 注意文件扩展名是.csv
    #output_file_path = f'F:\\data_all\\askubuntu_a2q\\K\\K_data_{i}.csv'
    input_file_path = f'G:\\data_all\\askubuntu_c2q\\N_data_{i}.csv'  # 注意文件扩展名是.csv
    output_file_path = f'G:\\data_all\\askubuntu_c2q\\K\\K_data_{i}.csv'
    
    df = pd.read_csv(input_file_path)
    
    k_values = []

    for index, row in df.iterrows():
        i = row['ego']
        j = row['alter']

        neighbors_i = len(df[df['ego'] == i]) + len(df[df['alter'] == i])
        neighbors_j = len(df[df['ego'] == j]) + len(df[df['alter'] == j])

        if neighbors_j == 0:
            neighbors_j = 1
  
        k = math.sqrt(neighbors_i * neighbors_j)

        k_values.append(k)

    df['weight'] = k_values

    df.to_csv(output_file_path, index=False)
    
    print(f'File {i} processed and saved to {output_file_path}')

File 300124 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_1.csv
File 335704 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_2.csv
File 482657 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_3.csv
File 482657 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_4.csv
File 443229 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_5.csv
File 515276 processed and saved to G:\data_all\askubuntu_c2q\K\K_data_6.csv


Age of tie

In [26]:
import pandas as pd

#user_contacts_df = pd.read_csv("F:\\data_all\\askubuntu_a2q\\filtered_results.csv")
user_contacts_df = pd.read_csv("G:\\data_all\\askubuntu_c2q\\filtered_results.csv")

valid_users_set = set(user_contacts_df['ego'].unique())

for file_index in range(1, 7):
    #input_file_path = f'F:\\data_all\\askubuntu_a2q\\data_{file_index}.csv'
    #output_file_path = f'F:\\data_all\\askubuntu_a2q\\T\\T_data_{file_index}.csv'
    input_file_path = f'G:\\data_all\\askubuntu_c2q\\data_{file_index}.csv'
    output_file_path = f'G:\\data_all\\askubuntu_c2q\\T\\T_data_{file_index}.csv'

    df = pd.read_csv(input_file_path)

    tij_values = []

    unique_user_pairs = df[['ego', 'alter']].drop_duplicates()

    for _, pair in unique_user_pairs.iterrows():
        i = pair['ego']
        j = pair['alter']

        if i in valid_users_set or j in valid_users_set:
            # Obtain the call records of the user pair
            user_pair_df = df[(df['ego'] == i) & (df['alter'] == j)]

            # Find the first and last call time of the user pair
            first_time = user_pair_df['time'].min()
            last_time = user_pair_df['time'].max()
            #first_time = user_pair_df['# timestamp'].min()
            #last_time = user_pair_df['# timestamp'].max()
            

            # Calculate the time difference, in days
            time_difference = (last_time - first_time) / (60 * 60 * 24)
            
            # Add the tij value to the list
            tij_values.append((i, j, time_difference))

    filtered_pairs = pd.DataFrame(tij_values, columns=['ego', 'alter', 'weight'])

    filtered_pairs.to_csv(output_file_path, index=False)
    
    print(f'File {file_index} processed and saved to {output_file_path}')

File 1 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_1.csv
File 2 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_2.csv
File 3 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_3.csv
File 4 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_4.csv
File 5 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_5.csv
File 6 processed and saved to G:\data_all\askubuntu_c2q\T\T_data_6.csv


Fraction of consecutive calls

In [27]:
import pandas as pd

for file_index in range(1, 7):
    #input_file_path = f'F:\\data_all\\askubuntu_a2q\\data_{file_index}.csv'
    #output_file_path =f'F:\\data_all\\askubuntu_a2q\\MU\\MU_data_{file_index}.csv'
    input_file_path = f'G:\\data_all\\askubuntu_c2q\\data_{file_index}.csv'
    output_file_path =f'G:\\data_all\\askubuntu_c2q\\MU\\MU_data_{file_index}.csv'

    df = pd.read_csv(input_file_path)

    # Initializes the empty list to store the numerator (number of calls satisfied) and denominator (total number of calls) of the μchatsij
    numerator_values = []
    denominator_values = []

    # Iterate over each pair of users
    unique_user_pairs = df[['ego', 'alter']].drop_duplicates()

    for _, pair in unique_user_pairs.iterrows():
        i = pair['ego']
        j = pair['alter']
        
        # Obtain the call records of the user pair
        user_pair_df = df[(df['ego'] == i) & (df['alter'] == j)]

        # Calculates the time between calls, in seconds
        #inter_event_times = user_pair_df['# timestamp'].diff()
        inter_event_times = user_pair_df['time'].diff()

        # Count the number of calls that meet the condition (<= 5 minutes)
        condition_met_count = (inter_event_times <= 5 * 60).sum()

        # The number of calls that meet the condition is added to the molecule
        numerator_values.append(condition_met_count)

        total_call_count = len(user_pair_df)

        denominator_values.append(total_call_count)

    mu_chats_values = [numerator / denominator if denominator != 0 else 0 for numerator, denominator in zip(numerator_values, denominator_values)]

    unique_user_pairs['weight'] = mu_chats_values

    unique_user_pairs.to_csv(output_file_path, index=False)
    
    print(f'File {file_index} processed and saved to {output_file_path}')

File 1 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_1.csv
File 2 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_2.csv
File 3 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_3.csv
File 4 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_4.csv
File 5 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_5.csv
File 6 processed and saved to G:\data_all\askubuntu_c2q\MU\MU_data_6.csv


Users’ Activity diversity

In [28]:
import pandas as pd
import math

for file_index in range(1, 7):
    #input_file_path = f'F:\\data_all\\askubuntu_a2q\\N\\N_data_{file_index}.csv'
    #output_file_path =f'F:\\data_all\\askubuntu_a2q\\A\\A_data_{file_index}.csv'
    input_file_path = f'G:\\data_all\\askubuntu_c2q\\N_data_{file_index}.csv'
    output_file_path =f'G:\\data_all\\askubuntu_c2q\\A\\A_data_{file_index}.csv'

    df = pd.read_csv(input_file_path)

    ki_kj_dict = {}

    for index, row in df.iterrows():
        i = row['ego']
        j = row['alter']

        neighbors_i = len(df[df['ego'] == i])+len(df[df['alter'] == i])

        neighbors_j = len(df[df['ego'] == j])+len(df[df['alter'] == j])
        
        if neighbors_j == 0:
            neighbors_j = 1
        
        ki_kj_dict[i] = neighbors_i
        ki_kj_dict[j] = neighbors_j

    user_i_counts = df.groupby('ego')['weight'].sum().reset_index()
    user_i_counts.columns = ['ego', 'ai']
    df = df.merge(user_i_counts, on='ego', how='left')

    def get_aj(user_j):
        if user_j in user_i_counts['ego'].values:
            aj = user_i_counts[user_i_counts['ego'] == user_j]['ai'].values[0]
        else:
            aj = 0 
        return aj

    # Use the apply function to compute aj for user j and create a new column
    df['aj'] = df['alter'].apply(get_aj)

    df['ki'] = df['ego'].map(ki_kj_dict)
    df['kj'] = df['alter'].map(ki_kj_dict)

    df['weight'] = (df['ai'] / df['ki']) * (df['aj'] / df['kj']).apply(math.sqrt)

    df.to_csv(output_file_path, index=False)
    
    print(f'File {file_index} processed and saved to {output_file_path}')

File 1 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_1.csv
File 2 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_2.csv
File 3 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_3.csv
File 4 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_4.csv
File 5 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_5.csv
File 6 processed and saved to G:\data_all\askubuntu_c2q\A\A_data_6.csv


In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import networkx as nx
import time
from collections import deque
import copy
"""Color the network visual module and find the node of the module"""

def Feasibility_weighted(G,G_motif,n1,n2,N1,node,hasEdge):
    """
    According to the isomorphism condition, determine whether it is the module to be searched
    :param G: Graph or (network), which can be directed or undirected, is networkx.Graph() or networkx.DiGraph()
    :param G_motif: The motif graph, can be directed or undirected but must be consistent with the graph G
    :param n1: Nodes in the motif network G_motif
    :param n2: Nodes in the motif network G_motif
    :param N1: Node in the target network G
    :param node: Node in the target network G
    :param hasEdge: Whether there is a positive edge between n1 and n2
    :return True or False: Represents whether it constitutes a motif or not
    """
    if hasEdge:
        if(G_motif[n1][n2]['weight'])!= (G[N1][node]['weight']):
            return False

    return True

def Feasibility_weighted_directed(G,G_motif,n1,n2,N1,node,hasEdge,hasEdge1):
    """
    According to the isomorphism condition, determine whether it is the module to be searched
    :param G: Graph or (network), which can be directed or undirected, is networkx.Graph() or networkx.DiGraph()
    :param G_motif: The motif graph, can be directed or undirected but must be consistent with the graph G
    :param n1: Nodes in the motif network G_motif
    :param n2: Nodes in the motif network G_motif
    :param N1: Node in the target network G
    :param node: Node in the target network G
    :param hasEdge: Whether there is a positive edge between n1 and n2
    :param hasEdge1: Whether there is an opposite edge between n2 and n1
    :return True or False: Represents whether it constitutes a motif or not
    """
    if hasEdge1:
        if (G_motif[n2][n1]['weight'])!= (G[node][N1]['weight']):
            return False

    if hasEdge:
        if (G_motif[n1][n2]['weight'])!= (G[N1][node]['weight']):
            return False

    return True

def preSort(G_motif,G2_node_list):
    """
    Preprocessing: Sort, adjust the search order, put the degree of the first, so that the more constraints, the more pruning, the faster the search.
    :param G_motif:
    :param G2_node_list:
    :param n_G2:
    """
    n_G2=len(G2_node_list)
    for i in range(1,n_G2):
        for j in range(0,n_G2-i):
            if G_motif.degree(G2_node_list[j+1])>G_motif.degree(G2_node_list[j]):
                t=G2_node_list[j]
                G2_node_list[j]=G2_node_list[j+1]
                G2_node_list[j+1]=t
    return G2_node_list

def DFS_nodes(M, m_nodes, rat, n, minQ, MS, deep, work, candi,result):

    if deep == len(m_nodes):
        if minQ >work:
            minQ=work
            for i in range(deep):
                result[i] = MS[i]
        # print(MS,minQ,work)
        return result,minQ
    work1=0
    candi1=0
    for node in m_nodes:
        if node in MS[:deep]:
            continue
        candi1 = candi
        for node1 in MS[:deep]:
            if M.has_edge(node,node1):
                candi1=rat*candi1
        candi1=candi1*n
        work1 = n * candi - candi1 + work
        MS[deep]=node
        result, minQ= DFS_nodes(M, m_nodes, rat, n, minQ, MS, deep + 1, work1, candi1, result)
    return result,minQ

def Motif_node_sort(M,m_nodes,deeps):
    """
    Find the fastest search order
    :param M:
    :param m_nodes:
    :param deeps:
    :return:
    """
    rat=0.5
    n=10
    minQ=99999999999999
    MS=['*' for x in range(len(m_nodes))]
    work=0
    candi=1
    m_nodes1=['*' for x in range(len(m_nodes))]
    if deeps==1:
        MS[0]=m_nodes[0]
    if deeps == 2:
        MS[0]=m_nodes[0]
        MS[1]=m_nodes[1]
    m_nodes1,minQ=DFS_nodes(M, m_nodes, rat, n, minQ, MS, deeps, work, candi,m_nodes1)
    return m_nodes1


def find_motif(G,Motif,G_node_list,G2_node_list,n_G2,MS,deeps,derepeat,weighted=False):
    # start = time.time()
    number = 0  
    deep = deeps
    SL=[]
    for i in range(n_G2-deeps):
        SL.append([])
        for j in range(0,n_G2-deeps-i):
            SL[i].append([])

    searchset = set(G_node_list)
    searchset = searchset - set(MS[0:deeps])
    for j in range(deeps, n_G2):
        SL[0][j - deeps] = searchset
    for i in range(deeps):
        ss = []
        neighbor1 = searchset & set(G.neighbors(MS[i]))
        neighbor2 = searchset - set(G.neighbors(MS[i]))
        for j in range(deeps, n_G2):
            hasEdge = Motif.has_edge(G2_node_list[i], G2_node_list[j])
            if (hasEdge):
                searchset1 = neighbor1
            else:
                searchset1 = neighbor2

            if weighted:
                for k in searchset1:
                    if (not Feasibility_weighted(G, Motif, G2_node_list[i], G2_node_list[j], MS[i], k,hasEdge)):
                        ss.append(k)
            searchset1=searchset1-set(ss)
            if (not searchset1):
                return 0
            if i == 0:
                SL[0][j-deeps] =  searchset1
            else:
                SL[0][j - deeps] = set(SL[0][j - deeps]) & searchset1

    if n_G2 - deeps < 2:
        return len(SL[0][0])
    Quelist = []
    for i in range(n_G2 - deeps):
        Quelist.append(deque())

    Quelist[deep - deeps].extend(SL[deep - deeps][0])
    ddd = False
    while (1):
        if deep == n_G2 - 1:
            number +=len(SL[deep-deeps][0])
            # print(MS, SL[deep - deeps][0]);
            deep -= 1


        while (not Quelist[deep - deeps]):
            deep -= 1
            if deep < deeps:
                return number

        MS[deep] = Quelist[deep - deeps].pop()
        neighbor1 = set(G.neighbors(MS[deep]))
        for i in range(1,n_G2-deep):
            searchset = SL[deep-deeps][i]
            searchset = searchset - set(MS[0:deep+1])
            hasEdge= Motif.has_edge(G2_node_list[deep], G2_node_list[deep+i])
            if (hasEdge):
                searchset = searchset & neighbor1
            else:
                searchset = searchset - neighbor1

            if (not searchset):
                ddd = False
                break
            if derepeat[deep+i]!=-1 and derepeat[deep+i]<=deep:

                rrr=[]
                for k in list(searchset):
                    if int(k) <int(MS[derepeat[deep+i]]):
                        rrr.append(k)
                searchset= searchset - set(rrr)
            ss=[]
            if weighted:
                for k in searchset:
                    if (not Feasibility_weighted(G, Motif, G2_node_list[deep], G2_node_list[deep+i], MS[deep], k,hasEdge)):
                        ss.append(k)
            searchset = searchset-set(ss)
            if (not searchset):
                ddd = False
                break
            else:
                SL[deep - deeps+1][i-1] = searchset
                ddd = True
        if ddd:
            deep += 1
            if deep < n_G2-1:
                Quelist[deep - deeps].extend(SL[deep-deeps][0])

def find_motif_directed(G,Motif,G_node_list,G2_node_list,n_G2,MS,deeps,derepeat,weighted=False):
    # start = time.time()
    # print("@@@@@",G2_node_list)
    edgelist = list(nx.edges(G))
    edgelist1 = []
    for i in edgelist:
        edgelist1.append((i[1], i[0]))

    G2 = nx.DiGraph()
    G2.add_nodes_from(G_node_list)
    G2.add_edges_from(edgelist1)

    number = 0  
    deep = deeps
    SL=[]
    for i in range(n_G2-deeps):
        SL.append([])
        for j in range(0,n_G2-deeps-i):
            SL[i].append([])

    searchset = set(G_node_list) - set(MS[0:deeps])
    for j in range(deeps, n_G2):
        SL[0][j - deeps] = searchset

    for i in range(deeps):
        ss = []
        neighbor1 = searchset & set(G.neighbors(MS[i]))
        neighbor2 = searchset - set(G.neighbors(MS[i]))
        neighbor3 = searchset & set(G2.neighbors(MS[i]))
        neighbor4 = searchset - set(G2.neighbors(MS[i]))
        for j in range(deeps, n_G2):
            hasEdge = Motif.has_edge(G2_node_list[i], G2_node_list[j])
            hasEdge1 = Motif.has_edge(G2_node_list[j], G2_node_list[i])
            if (hasEdge):
                searchset1 = neighbor1
            else:
                searchset1 = neighbor2
            if (hasEdge1):
                searchset1 = searchset1 & neighbor3
            else:
                searchset1 = searchset1 & neighbor4
            if weighted:
                for k in searchset1:
                    if ( not Feasibility_weighted_directed(G, Motif, G2_node_list[i], G2_node_list[j], MS[i], k,hasEdge,hasEdge1)):
                        ss.append(k)
            searchset1=searchset1-set(ss)
            if (not searchset1):
                return 0

            if i == 0:
                SL[0][j - deeps] = searchset1
            else:
                SL[0][j - deeps] = set(SL[0][j - deeps]) & searchset1

    if n_G2 - deeps < 2:
        return len(SL[0][0])
    Quelist = []
    for i in range(n_G2 - deeps - 1):
        Quelist.append(deque())

    Quelist[deep - deeps].extend(SL[deep - deeps][0])
    ddd = False
    while (1):
        if  deep == n_G2 - 1:
            number += len(SL[deep-deeps][0])
            deep -= 1
        while (not Quelist[deep - deeps]):
            deep -= 1
            if deep < deeps:
                return number
        
        MS[deep] = Quelist[deep - deeps].pop()
        neighbor1 = set(G.neighbors(MS[deep]))
        neighbor2 = set(G2.neighbors(MS[deep]))
        for i in range(1,n_G2-deep):
            searchset = SL[deep-deeps][i]
            searchset = searchset - set(MS[0:deep+1])
            hasEdge= Motif.has_edge(G2_node_list[deep], G2_node_list[deep+i])
            hasEdge1 = Motif.has_edge(G2_node_list[deep+i], G2_node_list[deep])
            if (hasEdge):
                searchset = searchset & neighbor1
            else:
                searchset = searchset - neighbor1
            if (hasEdge1):
                searchset = searchset & neighbor2
            else:
                searchset = searchset - neighbor2

            if (not searchset):
                ddd = False
                break

            if derepeat[deep + i] != -1 and derepeat[deep + i] <= deep:
                rrr = []
                for k in list(searchset):
                    if int(k) < int(MS[derepeat[deep + i]]):
                        rrr.append(k)
                searchset = searchset - set(rrr)

            ss=[]
            if weighted:
                for k in searchset:
                    if ( not Feasibility_weighted_directed(G, Motif, G2_node_list[deep], G2_node_list[deep+i], MS[deep], k,hasEdge,hasEdge1)):
                        ss.append(k)
            searchset=searchset-set(ss)
            if (not searchset):
                ddd = False
                break
            else:
                SL[deep - deeps+1][i-1] = searchset
                ddd = True
        if ddd:
            deep += 1
            if deep < n_G2-1:
                Quelist[deep - deeps].extend(SL[deep-deeps][0])



def combine_motif_list(motif_list,MS):
    ll=copy.deepcopy(list(MS))
    motif_list.append(ll)
    # print(motif_list)

def filter_greater(node_list,num1):
    result=[]
    for i in node_list:
        if int(i) > num1:
            result.append(i)
    return result

def find_motif_list(G,Motif,G_node_list,G2_node_list,n_G2,MS,deeps,derepeat,weighted=False):
    # start = time.time()
    motif_list =[]
    number = 0  
    deep = deeps
    SL=[]
    for i in range(n_G2-deeps):
        SL.append([])
        for j in range(0,n_G2-deeps-i+1):
            SL[i].append([])

    searchset = set(G_node_list)
    searchset = searchset - set(MS[0:deeps])
    for j in range(deeps, n_G2):
        SL[0][j - deeps] = searchset

    for i in range(deeps):
        ss = []
        neighbor1 = searchset & set(G.neighbors(MS[i]))
        neighbor2 = searchset - set(G.neighbors(MS[i]))
        for j in range(deeps, n_G2):
            hasEdge=Motif.has_edge(G2_node_list[i], G2_node_list[j])
            if (hasEdge):
                searchset1 = neighbor1
            else:
                searchset1 = neighbor2

            if weighted:
                for k in searchset1:
                    if (not Feasibility_weighted(G, Motif, G2_node_list[i], G2_node_list[j], MS[i], k,hasEdge)):
                        ss.append(k)
                searchset1=searchset1-set(ss)
            if (not searchset1):
                return motif_list
            if i == 0:
                SL[0][j-deeps] =  searchset1
            else:
                SL[0][j - deeps] = set(SL[0][j - deeps]) & searchset1

    if n_G2 - deeps < 2:
        for i in SL[0][0]:
            MS[n_G2-1]=i
            combine_motif_list(motif_list, MS)
        return motif_list
    Quelist = []
    for i in range(n_G2 - deeps):
        Quelist.append(deque())

    Quelist[deep - deeps].extend(SL[deep - deeps][0])
    ddd = False

    while (1):
        # print("++++++",motif_list, MS)
        if deep > n_G2 - 1:
            deep -= 1
            # if derepeat[deep] !=-1:
            #     print("++++++++++++++",deep)
            #     SL[deep-deeps][0]=filter_greater(SL[deep-deeps][0],int(MS[derepeat[deep]]))
            number += len(SL[deep-deeps][0])
            combine_motif_list(motif_list,MS)

        while (not Quelist[deep - deeps]):
            deep -= 1
            if deep < deeps:
                return motif_list

        if derepeat[deep] != -1:
            MS[deep] = Quelist[deep - deeps].pop()
            while(int(MS[deep]) < int(MS[derepeat[deep]])):
                while (not Quelist[deep - deeps]):
                    deep -= 1
                    if deep < deeps:
                        return motif_list
                MS[deep] = Quelist[deep - deeps].pop()
                if derepeat[deep] == -1:
                    break
        else:
            MS[deep] = Quelist[deep - deeps].pop()
        neighbor1 = set(G.neighbors(MS[deep]))
        for i in range(1,n_G2-deep):
            searchset = set(SL[deep-deeps][i])
            searchset = searchset - set(MS[0:deep+1])
            hasEdge= Motif.has_edge(G2_node_list[deep], G2_node_list[deep+i])
            if (hasEdge):
                searchset = searchset & neighbor1
            else:
                searchset = searchset - neighbor1

            if (not searchset):
                ddd = False
                break
            ss=[]

            if weighted:
                for k in searchset:
                    if (not Feasibility_weighted(G, Motif, G2_node_list[deep], G2_node_list[deep+i], MS[deep], k,hasEdge)):
                        ss.append(k)
            searchset=searchset-set(ss)
            if (not searchset):
                ddd = False
                break
            else:
                SL[deep - deeps+1][i-1] = searchset
                ddd = True
        if ddd:
            deep += 1
            if deep < n_G2:
                Quelist[deep - deeps].extend(SL[deep-deeps][0])

def find_motif_directed_list(G,Motif,G_node_list,G2_node_list,n_G2,MS,deeps,derepeat,weighted=False):
    # start = time.time()
    # print("@@@@@",G2_node_list)
    motif_list = []
    edgelist = list(nx.edges(G))
    edgelist1 = []
    for i in edgelist:
        edgelist1.append((i[1], i[0]))
    G2 = nx.DiGraph()
    G2.add_nodes_from(G_node_list)
    G2.add_edges_from(edgelist1)

    number = 0  
    deep = deeps
    SL=[]
    for i in range(n_G2-deeps):
        SL.append([])
        for j in range(0,n_G2-deeps-i+1):
            SL[i].append([])
    searchset = set(G_node_list) - set(MS[0:deeps])
    for j in range(deeps, n_G2):
        SL[0][j - deeps] = searchset

    for i in range(deeps):

        ss = []
        neighbor1 = searchset & set(G.neighbors(MS[i]))
        neighbor2 = searchset - set(G.neighbors(MS[i]))
        neighbor3 = searchset & set(G2.neighbors(MS[i]))
        neighbor4 = searchset - set(G2.neighbors(MS[i]))
        for j in range(deeps, n_G2):
            hasEdge=Motif.has_edge(G2_node_list[i], G2_node_list[j])
            hasEdge1 = Motif.has_edge(G2_node_list[j], G2_node_list[i])
            if (hasEdge):
                searchset1 = neighbor1
            else:
                searchset1 = neighbor2
            if (hasEdge1):
                searchset1 = searchset1 & neighbor3
            else:
                searchset1 = searchset1 & neighbor4
            if weighted:
                for k in searchset1:
                    if ( not Feasibility_weighted_directed(G, Motif, G2_node_list[i], G2_node_list[j], MS[i], k,hasEdge,hasEdge1)):
                        ss.append(k)
            searchset1=searchset1-set(ss)
            if (not searchset1):
                return motif_list

            if i == 0:
                SL[0][j-deeps]=  searchset1
            else:
                SL[0][j - deeps] = set(SL[0][j - deeps]) & searchset1

    if n_G2 - deeps < 2:
        for i in SL[0][0]:
            MS[n_G2-1]=i
            combine_motif_list(motif_list, MS)

        return motif_list
        # return SL[0][0]
    Quelist = []
    for i in range(n_G2 - deeps):
        Quelist.append(deque())

    Quelist[deep - deeps].extend(SL[deep - deeps][0])
    ddd = False

    while (1):
        if  deep > n_G2 - 1:
            deep -= 1
            # if derepeat[deep] !=-1:
            #     print("++++++++++++++",deep)
            #     SL[deep-deeps][0]=filter_greater(SL[deep-deeps][0],int(MS[derepeat[deep]]))
            number += len(SL[deep-deeps][0])
            combine_motif_list(motif_list, MS)
        while (not Quelist[deep - deeps]):
            deep -= 1
            if deep < deeps:
                return motif_list

        if derepeat[deep] != -1:
            MS[deep] = Quelist[deep - deeps].pop()

            while(int(MS[deep]) < int(MS[derepeat[deep]])):
                while (not Quelist[deep - deeps]):
                    deep -= 1
                    if deep < deeps:
                        return number
                MS[deep] = Quelist[deep - deeps].pop()
                if derepeat[deep] == -1:
                    break
        else:
            MS[deep] = Quelist[deep - deeps].pop()
        neighbor1 = set(G.neighbors(MS[deep]))
        neighbor2 = set(G2.neighbors(MS[deep]))
        for i in range(1,n_G2-deep):
            searchset = SL[deep-deeps][i]
            searchset = searchset - set(MS[0:deep+1])
            hasEdge= Motif.has_edge(G2_node_list[deep], G2_node_list[deep+i])
            hasEdge1 = Motif.has_edge(G2_node_list[deep+i], G2_node_list[deep])
            if (hasEdge):
                searchset = searchset & neighbor1
            else:
                searchset = searchset - neighbor1
            if (hasEdge1):
                searchset = searchset & neighbor2
            else:
                searchset = searchset - neighbor2

            if (not searchset):
                ddd = False
                break
            ss=[]
            if weighted:
                for k in searchset:
                    if ( not Feasibility_weighted_directed(G, Motif, G2_node_list[deep], G2_node_list[deep+i], MS[deep], k,hasEdge,hasEdge1)):
                        ss.append(k)
            searchset=searchset-set(ss)
            if (not searchset):
                ddd = False
                break
            else:
                SL[deep - deeps+1][i-1] = searchset
                ddd = True
        if ddd:
            deep += 1
            if deep < n_G2:
                Quelist[deep - deeps].extend(SL[deep-deeps][0])


def get_node_group(G2_node_list,n_G2,repetitions):

    nodes_group={}
    for i in G2_node_list:
        nodes_group[i]={}
        for j in G2_node_list:
            nodes_group[i][j]=-1
    for i in range(n_G2):
        for j in range(len(repetitions)):
            for k in range(len(repetitions)):
                nodes_group[repetitions[j][i]][repetitions[k][i]] = 1
                nodes_group[repetitions[k][i]][repetitions[j][i]] = 1
    return nodes_group

def Build_Motif_Tree(motif, nodelist):
    motif_tree={}
    mark={}
    # nodelist=list(nx.nodes(motif))
    for i in nodelist:
        mark[i]=0
        motif_tree[i]=[]
    mark[nodelist[0]]=1
    build_tree(motif,mark,nodelist[0],motif_tree)
    return motif_tree

def build_tree(motif,mark,node,motif_tree):
    a=list(motif.neighbors(node))
    b=[]
    for i in a:
        if mark[i] ==0:
            b.append(i)
            mark[i]=1
    # if b==[]:
    #     return
    motif_tree[node]=b
    for i in motif_tree[node]:
        build_tree(motif,mark,i,motif_tree)


def get_prevent_repetition_list(motif,G2_node_list,len1,repetitions):
    isconnect=True
    direc=nx.is_directed(motif)
    if not direc:
        isconnect = nx.is_connected(motif)
    if( (not isconnect) and (not direc)):
        moti = nx.Graph()
        # edge_ss=[]
        moti.add_nodes_from(G2_node_list)
        for i in G2_node_list:
            for j in G2_node_list:
                if(i!=j):
                    if( not motif.has_edge(i,j)):
                        moti.add_edge(i,j)
                        # edge_ss.append((i,j))
                    if( not motif.has_edge(j,i)):
                        moti.add_edge(j, i)
                        # edge_ss.append((j, i))
        # moti.add_edges_from(edge_ss)
        motif = moti


    nodes_group = get_node_group(G2_node_list,len1,repetitions)

    motif_tree = Build_Motif_Tree(motif,G2_node_list)

    mapnodelist={}
    for i in range(len(G2_node_list)):
        mapnodelist[G2_node_list[i]]=i

    derepet = [-1 for i in range(len1)]

    for i in G2_node_list[1:]:
        if nodes_group[G2_node_list[0]][i]==1:
            derepet[mapnodelist[i]]=0

    mark={}
    for i in G2_node_list:
        mark[i]=0
    for node in G2_node_list:
        for i in range(len(motif_tree[node])):
            if mark[motif_tree[node][i]]== 0:
                aaa=[]
                aaa.append(motif_tree[node][i])
                mark[motif_tree[node][i]] = 1
                for j in range(len(motif_tree[node])):
                    if i < j:
                        if nodes_group[motif_tree[node][i]][motif_tree[node][j]] == 1:
                            aaa.append(motif_tree[node][j])
                            mark[motif_tree[node][j]]=1
                bbb=[]
                for k in G2_node_list:
                    if k in aaa:
                        bbb.append(k)
                for k in range(len(bbb)):
                    if k==0:
                        continue
                    derepet[mapnodelist[bbb[k]]]=mapnodelist[bbb[k-1]]


    return derepet


def node_motif_num(G,G_motif,node,orbit_node,directed=False,weighted=False):
    """
        Calculate the number of motifs that node nodes participate in as the orbit of the motif
        :param G: Graph or (network), which can be directed or undirected, is networkx.Graph() or networkx.DiGraph()
        :param G_motif: The motif graph, can be directed or undirected but must be consistent with the graph G
        :param node: A node in the graph G
        :param orbit_node: A node in the motif represents a track of the motif
        :param directed: Whether it is a directed graph. If it is True, the directed graph value is False. The default value is undirected graph False
        :param weighted: If the value is True, if the value is False, the default is False. The symbolic network is regarded as a weighted network.
        :return :Dot the number of motifs
    """

    # start = time.time()
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    len1 = len(G2_node_list)
    # Adjust the node order according to the track
    for nod in range(len1):
        if G2_node_list[nod] == orbit_node:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[0]
    G2_node_list[0] = orbit_node


    # G2_node_list[1:len1]=preSort(G_motif, G2_node_list[1:len1])  
    G2_node_list = Motif_node_sort(G_motif, G2_node_list, 1)
    MS = ['*' for x in range(len1)]
    node_motif_number = 0
    if directed:
        MS[0] = G2_node_list[0]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        MS[0] = node
        node_motif_number = find_motif_directed(G, G_motif, G_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        MS[0] = G2_node_list[0]
        # a = find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
    else:
        MS[0] = G2_node_list[0]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        MS[0] = node
        node_motif_number = find_motif(G, G_motif, G_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        MS[0] = G2_node_list[0]
    return node_motif_number

def edge_motif_num(G,G_motif,edge,orbit_edge,directed=False,weighted=False):
    """
    Calculate the number of motifs that the edge participates in as the orbit of the motif:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param edge: An edge in figure G
    :param orbit_edge: An edge in a motif represents an edge orbit of the motif:param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: The return value is the number of edge motifs
    """
    
    twoWay = False
    if (not G.has_edge(edge[0], edge[1])) and (not G.has_edge(edge[1], edge[0])):
        print("There is no such edge in the network.")
        twoWay = True
    if G.has_edge(edge[1], edge[0]) and G.has_edge(edge[0], edge[1]):
        twoWay = True

    # start = time.time()
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    len1 = len(G2_node_list)

    for nod in range(len1):
        if G2_node_list[nod] == orbit_edge[0]:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[0]
    G2_node_list[0] = orbit_edge[0]
    for nod in range(len1):
        if G2_node_list[nod] == orbit_edge[1]:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[1]
    G2_node_list[1] = orbit_edge[1]


    # G2_node_list[2:len1]=preSort(G_motif, G2_node_list[2:len1])  # 调整搜索顺序
    G2_node_list = Motif_node_sort(G_motif, G2_node_list, 2)
    MS = ['*' for x in range(len1)]
    edge_motif_number=0

    if directed:

        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list,len1, repetitions)
        if (derepeat[1] != -1):
            twoWay = False
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat,weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)

        MS[0] = edge[0]
        MS[1] = edge[1]
        edge_motif_number = find_motif_directed(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        # a = find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        if twoWay:  # 如果是双向边还需要查找反方向的模体数
            MS[0] = edge[1]
            MS[1] = edge[0]
            edge_motif_number += find_motif_directed(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
            MS[0] = G2_node_list[1]
            MS[1] = G2_node_list[0]
            # a += find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)

    else:

        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        if (derepeat[1] != -1):
            twoWay = False

        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        #------------------------------#
        MS[0] = edge[0]
        MS[1] = edge[1]
        edge_motif_number= find_motif(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        # a = find_motif(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        if twoWay: 
            MS[0] = edge[1]
            MS[1] = edge[0]
            edge_motif_number += find_motif(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat,weighted)
            MS[0] = G2_node_list[1]
            MS[1] = G2_node_list[0]
    return edge_motif_number


def total_motif_num(G,G_motif,directed=False,weighted=False):
    """
    Calculate the total number of motifs:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: The motif graph can be directed or undirected but must be of the same type as the graph G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: The total number of motifs is returned
    """
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    n_G2 = len(G2_node_list)
    # start = time.time()

    # G2_node_list=preSort(G_motif, G2_node_list) 
    G2_node_list = Motif_node_sort(G_motif, G2_node_list,0)

    MS = ['*' for x in range(n_G2)]
    total_motif_number = 0
    # repetitions = 0
    if directed:
        derepeat = [-1 for i in range(n_G2)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, n_G2, repetitions)
        total_motif_number = find_motif_directed(G, G_motif, G_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)

    else:
        derepeat = [-1 for i in range(n_G2)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, n_G2, MS, 0,derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, n_G2, repetitions)
        # print(derepeat)
        total_motif_number = find_motif(G, G_motif, G_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)
    return total_motif_number



def total_motif_list(G,G_motif,directed=False,weighted=False):
    """
    Calculate the total number of motifs:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: The motif graph can be directed or undirected but must be of the same type as the graph G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: The total number of motifs is returned
    """
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    n_G2 = len(G2_node_list)
    # start = time.time()

    # G2_node_list=preSort(G_motif, G2_node_list) 
    G2_node_list = Motif_node_sort(G_motif, G2_node_list,0)

    MS = ['*' for x in range(n_G2)]


    if directed:
        derepeat = [-1 for i in range(n_G2)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, n_G2, repetitions)
        total_motif = find_motif_directed_list(G, G_motif, G_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)
        # a=find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)

    else:
        derepeat = [-1 for i in range(n_G2)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, n_G2, MS, 0,derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, n_G2, repetitions)
        # print(derepeat)
        total_motif = find_motif_list(G, G_motif, G_node_list, G2_node_list, n_G2, MS, 0, derepeat, weighted)

    motifss = total_motif

    motif_edge=[]
    for i in range(len(motifss)):
        motif_edge.append([])
    if directed:
        for i in range(len(G2_node_list)):
            for j in  range(len(G2_node_list)):
                if(i!=j):
                    if(G_motif.has_edge(G2_node_list[i],G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i],motifss[k][j]))
    else:
        for i in range(len(G2_node_list)):
            for j in range(len(G2_node_list)):
                if (j>i):
                    if(G_motif.has_edge(G2_node_list[i],G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i],motifss[k][j]))
    return motifss,motif_edge


def node_orbit(G,directed=False,weighted=False):
    node_list1 = list(G.nodes)
    node_num = len(node_list1)
    # start = time.time()
    MS = ['*' for x in range(node_num)]
    derepeat = [-1 for i in range(node_num)]
    if directed:
        repetitions = find_motif_directed_list(G, G, node_list1, node_list1, node_num, MS, 0, derepeat,weighted)
    else:
        repetitions = find_motif_list(G, G, node_list1, node_list1, node_num, MS, 0, derepeat, weighted)


    repelen=len(repetitions)
    orbit_list=[]

    sum=0
    nodesss=[]
    for i in range(node_num):
        orbit = set()
        if (repetitions[0][i] in nodesss):
            continue
        for j in range(repelen):
            orbit.add(repetitions[j][i])
            nodesss.append(repetitions[j][i])
        sum=sum+len(orbit)
        orbit_list.append(list(orbit))
        if(sum >= node_num):
            break
    # print(orbit_list)
    return orbit_list

def edge_orbit(G,directed=False,weighted=False):
    edge_list=list(G.edges)
    edge_num=len(edge_list)
    edge2node= range(edge_num)
    node2edge=[]
    G2 = nx.Graph()
    node_list1 = list(G.nodes)
    node_num = len(node_list1)
    if directed or weighted:
        # start = time.time()
        MS = ['*' for x in range(node_num)]
        derepeat = [-1 for i in range(node_num)]
        if directed:
            repetitions = find_motif_directed_list(G, G, node_list1, node_list1, node_num, MS, 0, derepeat, weighted)
        else:
            repetitions = find_motif_list(G, G, node_list1, node_list1, node_num, MS, 0, derepeat, weighted)

        repelen = len(repetitions)
        orbit_list = []

        sum = 0
        nodesss = []
        for i in range(node_num-1):
            for j in range(i+1,node_num):
                if(G.has_edge(node_list1[i],node_list1[j]) or G.has_edge(node_list1[j],node_list1[i])):
                    orbit = set()
                    if ((repetitions[0][i],repetitions[0][j]) in nodesss):
                        continue
                    for k in range(repelen):
                        orbit.add((repetitions[k][i],repetitions[k][j]))
                        nodesss.append((repetitions[k][i],repetitions[k][j]))
                    sum = sum + len(orbit)
                    orbit_list.append(list(orbit))
                    if (sum >= edge_num):
                        break
        # print(orbit_list)
        return orbit_list
    else:
        for i in edge2node:
            for j in edge2node:
                if i >= j:
                    continue
                if edge_list[i][0] == edge_list[j][0]:
                    node2edge.append((i, j))
                if edge_list[i][0] == edge_list[j][1]:
                    node2edge.append((i, j))
                if edge_list[i][1] == edge_list[j][0]:
                    node2edge.append((i, j))
                if edge_list[i][1] == edge_list[j][1]:
                    node2edge.append((i, j))


        G2.add_edges_from(node2edge)
        orbit_list= node_orbit(G2)
        result=[]
        for orbit in range(len(orbit_list)):
            result.append([])
            for i in orbit_list[orbit]:
                result[orbit].append(edge_list[i])

        return result



def node_in_motif(G,G_motif,node,directed=False,weighted=False):
    """
    The number of motifs involved by node is counted:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param node: indicates a node in Figure G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: Number of dot motifs
    """
    result=0
    orbit_list=node_orbit(G_motif)
    for i in range(len(orbit_list)):
        # start = time.time()
        result += node_motif_num(G, G_motif, node,orbit_list[i][0], directed, weighted)  # 点模体
    print("开始运行")
    return result

def edge_in_motif(G,G_motif,edge,directed=False,weighted=False):
    """
    Count the number of edges participating in constituting the motif:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param edge: An edge in figure G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: Number of dot motifs
    """
    result=0
    orbit_list=edge_orbit(G_motif)
    for i in range(len(orbit_list)):
        # start = time.time()
        result += edge_motif_num(G, G_motif, edge,orbit_list[i][0], directed, weighted)  # 点模体
    return result


def node_coverage_rate_of_motif(G,G_motif,directed=False, weighted=False):
    G_node_list = list(nx.nodes(G))
    node_num=len(G_node_list)
    in_motif=0
    for node in G_node_list:
        a = node_in_motif(G, G_motif, node, directed, weighted)
        if a>0:
            in_motif+=1
    return in_motif/node_num

def edge_coverage_rate_of_motif(G,G_motif,directed=False, weighted=False):
    G_edge_list = list(nx.edges(G))
    edge_num = len(G_edge_list)
    in_motif = 0
    for edge in G_edge_list:
        a = edge_in_motif(G, G_motif, edge, directed, weighted)
        if a > 0:
            in_motif += 1
    return in_motif / edge_num


def node_motif_list(G,G_motif,node,orbit_node,directed=False,weighted=False):
    """
    Calculate the set of motifs in which node participates:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param node: indicates a node in Figure G
    :param orbit_node: A node in a motif represents an orbit of the motif:param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: A collection of point motifs
    """

    # start = time.time()
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    len1 = len(G2_node_list)
    for nod in range(len1):
        if G2_node_list[nod] == orbit_node:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[0]
    G2_node_list[0] = orbit_node

    G2_node_list = Motif_node_sort(G_motif, G2_node_list, 1)
    MS = ['*' for x in range(len1)]
    node_motif_number = 0
    if directed:

        MS[0] = G2_node_list[0]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)

        MS[0] = node
        node_motif = find_motif_directed_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        MS[0] = G2_node_list[0]
        # a = find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
    else:
        MS[0] = G2_node_list[0]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        # print(repetitions)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        # print(derepeat)
        MS[0] = node
        node_motif = find_motif_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 1, derepeat, weighted)
        MS[0] = G2_node_list[0]

    motifss = node_motif

    # print("motif num :", len(motifss))
    motif_edge = []
    for i in range(len(motifss)):
        motif_edge.append([])
    if directed:
        for i in range(len(G2_node_list)):
            for j in range(len(G2_node_list)):
                if (i != j):
                    if (G_motif.has_edge(G2_node_list[i], G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i], motifss[k][j]))
    else:
        for i in range(len(G2_node_list)):
            for j in range(len(G2_node_list)):
                if (j > i):
                    if (G_motif.has_edge(G2_node_list[i], G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i], motifss[k][j]))
    # print(motif_edge)
    return motifss, motif_edge

def edge_motif_list(G,G_motif,edge,orbit_edge,directed=False,weighted=False):
    """
    Calculate the set of motifs in which edge participates:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param edge: An edge in figure G
    :param orbit_edge: An edge in a motif represents an edge orbit of the motif:param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: The return value is a collection of edge motifs
    """

    twoWay = False
    if (not G.has_edge(edge[0], edge[1])) and (not G.has_edge(edge[1], edge[0])):
        print("There is no such edge in the network.")
        twoWay = True
    if G.has_edge(edge[1], edge[0]) and G.has_edge(edge[0], edge[1]):
        twoWay = True


    # start = time.time()
    G_node_list = list(nx.nodes(G))
    G2_node_list = list(nx.nodes(G_motif))
    len1 = len(G2_node_list)

    for nod in range(len1):
        if G2_node_list[nod] == orbit_edge[0]:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[0]
    G2_node_list[0] = orbit_edge[0]
    for nod in range(len1):
        if G2_node_list[nod] == orbit_edge[1]:
            ab = nod
            break
    G2_node_list[ab] = G2_node_list[1]
    G2_node_list[1] = orbit_edge[1]


    # G2_node_list[2:len1]=preSort(G_motif, G2_node_list[2:len1])
    G2_node_list = Motif_node_sort(G_motif, G2_node_list, 2)
    MS = ['*' for x in range(len1)]
    edge_motif_number=0


    if directed:

        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list,len1, repetitions)
        if (derepeat[1] != -1):
            twoWay = False

        MS[1] = G2_node_list[1]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_directed_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat,weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)

        MS[0] = edge[0]
        MS[1] = edge[1]
        edge_motif = find_motif_directed_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        # a = find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        if twoWay: 
            MS[0] = edge[1]
            MS[1] = edge[0]
            edge_motif.extend( find_motif_directed_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted))
            MS[0] = G2_node_list[1]
            MS[1] = G2_node_list[0]
            # a += find_motif_directed(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)

    else:

        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 0, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)

        if (derepeat[1] != -1):
            twoWay = False
        derepeat = [-1 for i in range(len1)]
        repetitions = find_motif_list(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        derepeat = get_prevent_repetition_list(G_motif, G2_node_list, len1, repetitions)
        #------------------------------#
        MS[0] = edge[0]
        MS[1] = edge[1]
        edge_motif= find_motif_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        MS[0] = G2_node_list[0]
        MS[1] = G2_node_list[1]
        # a = find_motif(G_motif, G_motif, G2_node_list, G2_node_list, len1, MS, 2, derepeat, weighted)
        if twoWay:  
            MS[0] = edge[1]
            MS[1] = edge[0]
            edge_motif.extend(  find_motif_list(G, G_motif, G_node_list, G2_node_list, len1, MS, 2, derepeat,weighted))
            MS[0] = G2_node_list[1]
            MS[1] = G2_node_list[0]

    motifss = edge_motif

    # print("motif num :", len(motifss))
    motif_edge = []
    for i in range(len(motifss)):
        motif_edge.append([])
    if directed:
        for i in range(len(G2_node_list)):
            for j in range(len(G2_node_list)):
                if (i != j):
                    if (G_motif.has_edge(G2_node_list[i], G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i], motifss[k][j]))
    else:
        for i in range(len(G2_node_list)):
            for j in range(len(G2_node_list)):
                if (j > i):
                    if (G_motif.has_edge(G2_node_list[i], G2_node_list[j])):
                        for k in range(len(motifss)):
                            motif_edge[k].append((motifss[k][i], motifss[k][j]))
    # print(motif_edge)
    return motifss, motif_edge


def node_in_motif_list(G,G_motif,node,directed=False,weighted=False):
    """
    The number of motifs involved by node is counted:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param node: indicates a node in Figure G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: Number of dot motifs
    """
    result_node = []
    result_edge = []
    orbit_list=node_orbit(G_motif)
    for i in range(len(orbit_list)):
        # start = time.time()
        nodes, edges =node_motif_list(G, G_motif, node,orbit_list[i][0], directed, weighted)
        result_node.extend(nodes) 
        result_edge.extend(edges)
    return result_node,result_edge

def edge_in_motif_list(G,G_motif,edge,directed=False,weighted=False):
    """
    Count the number of edges participating in constituting the motif:param G: graph or (network), which can be directed or undirected, networkx.Graph() or networkx.DiGraph()
    :param G_motif: A motif graph that can be directed or undirected but must be consistent with the graph G
    :param edge: An edge in figure G
    :param directed: Indicates whether the directed graph is True, otherwise the directed graph is False, and the undirected graph is False by default
    :param weighted: indicates whether the network has a weight. If the value is True, if the value is False, the default value is False. The symbolic network is regarded as a weighted network.
    :return: Number of dot motifs
    """
    result_node=[]
    result_edge=[]
    orbit_list=edge_orbit(G_motif)
    for i in range(len(orbit_list)):
        # start = time.time()
        nodes,edges=edge_motif_list(G, G_motif, edge, orbit_list[i][0], directed, weighted)
        result_node.extend(nodes)
        result_edge.extend(edges)
    return result_node,result_edge

In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# node_in_motif function in motif.ipynb

for file_index in range(1, 7):
    input_file_path = f'F:\\data_all\\superuser_c2a\\N\\N_data_{file_index}.csv'
    output_file_path =f'F:\\data_all\\superuser_c2a\\M\\M9_data_{file_index}.csv'

    df = pd.read_csv(input_file_path)
    ego = df["ego"]
    alter = df["alter"]
    ego_list = list(set(list(ego)).union(set(list(alter))))
    edge = [(ego[i], alter[i]) for i in range(len(ego))]

    G = nx.Graph()
    G.add_nodes_from(ego_list)
    G.add_edges_from(edge)

    M = []
    df1 = pd.read_csv("F:\\data_all\\superuser_c2a\\filtered_results.csv")
    alter1 = df1["ego"]

    for i in range(len(alter1)):
        print("开始执行")
        #g1 = nx.Graph()
        #g1.add_nodes_from([1, 2, 3, 4])
        #g1.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 4), (2, 3), (3, 4)])  # M1
        #number1 = node_in_motif(G, g1, alter1[i], directed=False, weighted=False)
        #M.append(number1)
        #g2 = nx.Graph()
        #g2.add_nodes_from([1, 2, 3, 4])
        #g2.add_edges_from([(1, 2), (1, 3), (1, 4), (2,4),(2,3)]) # M2
        #number2 = node_in_motif(G, g2, alter1[i], directed=False, weighted=False)
        #M.append(number2)
        #g3 = nx.Graph()
        #g3.add_nodes_from([1, 2, 3, 4])
        #g3.add_edges_from([(1, 2), (2, 3), (3, 4), (4,1)]) # M3
        #number3 = node_in_motif(G, g3, alter1[i], directed=False, weighted=False)
        #M.append(number3)
        #g4 = nx.Graph()
        #g4.add_nodes_from([1, 2, 3, 4])
        #g4.add_edges_from([(1, 3), (1, 4), (3,4),(2,3)]) # M4
        #number4 = node_in_motif(G, g4, alter1[i], directed=False, weighted=False)
        #M.append(number4)
        #g5 = nx.Graph()
        #g5.add_nodes_from([1, 2, 3, 4])
        #g5.add_edges_from([(1, 2), (4, 3), (1, 4)]) # M5
        #number5 = node_in_motif(G, g5, alter1[i], directed=False, weighted=False)
        #M.append(number5)
        #g6 = nx.Graph()
        #g6.add_nodes_from([1, 2, 3, 4])
        #g6.add_edges_from([(1, 3),(3,4),(2,3)]) # M6
        #number6 = node_in_motif(G, g6, alter1[i], directed=False, weighted=False)
        #M.append(number6)
        #g7 = nx.Graph()
        #g7.add_nodes_from([1, 2, 3, 4])
        #g7.add_edges_from([(1, 3), (3, 4), (1,4)]) # M7
        #number7 = node_in_motif(G, g7, alter1[i], directed=False, weighted=False)
        #M.append(number7)
        #g8 = nx.Graph()
        #g8.add_nodes_from([1, 2, 3, 4])
        #g8.add_edges_from([(1, 2), (3, 4)]) # M8
        #number8 = node_in_motif(G, g8, alter1[i], directed=False, weighted=False)
        #M.append(number8)
        g9 = nx.Graph()
        g9.add_nodes_from([1, 2, 3, 4])
        g9.add_edges_from([(3, 4), (1,4)]) # M9
        number9 = node_in_motif(G, g9, alter1[i], directed=False, weighted=False)
        M.append(number9)
        #g10 = nx.Graph()
        #g10.add_nodes_from([1, 2, 3, 4])
        #g10.add_edges_from([(3, 4)])
        #number10 = node_in_motif(G, g10, alter1[i], directed=False, weighted=False)
        #M.append(number10)
        #g11 = nx.Graph()
        #g11.add_nodes_from([1, 2, 3, 4])
        #g11.add_edges_from([]) # M11
        #number11 = node_in_motif(G, g11, alter1[i], directed=False, weighted=False)
        #M.append(number11)
        print(f"第{i+1}次结果")
    M_list = pd.DataFrame(M)
    M_list.to_csv(output_file_path)
    print(f"File {file_index} processed and saved to {output_file_path}")

开始执行
开始运行
第1次结果
开始执行
开始运行
第2次结果
开始执行
开始运行
第3次结果
开始执行
开始运行
第4次结果
开始执行
开始运行
第5次结果
开始执行
开始运行
第6次结果
开始执行
开始运行
第7次结果
开始执行
开始运行
第8次结果
开始执行
开始运行
第9次结果
开始执行
开始运行
第10次结果
开始执行
开始运行
第11次结果
开始执行
开始运行
第12次结果
开始执行
开始运行
第13次结果
File 1 processed and saved to F:\data_all\superuser_c2a\M\M9_data_1.csv
开始执行
开始运行
第1次结果
开始执行
开始运行
第2次结果
开始执行
开始运行
第3次结果
开始执行
开始运行
第4次结果
开始执行
开始运行
第5次结果
开始执行
开始运行
第6次结果
开始执行
开始运行
第7次结果
开始执行
开始运行
第8次结果
开始执行
开始运行
第9次结果
开始执行
开始运行
第10次结果
开始执行
开始运行
第11次结果
开始执行
开始运行
第12次结果
开始执行
开始运行
第13次结果
File 2 processed and saved to F:\data_all\superuser_c2a\M\M9_data_2.csv
开始执行
开始运行
第1次结果
开始执行
开始运行
第2次结果
开始执行
开始运行
第3次结果
开始执行
开始运行
第4次结果
开始执行
开始运行
第5次结果
开始执行
开始运行
第6次结果
开始执行
开始运行
第7次结果
开始执行
开始运行
第8次结果
开始执行
开始运行
第9次结果
开始执行
开始运行
第10次结果
开始执行
开始运行
第11次结果
开始执行
开始运行
第12次结果
开始执行
开始运行
第13次结果
File 3 processed and saved to F:\data_all\superuser_c2a\M\M9_data_3.csv
开始执行
开始运行
第1次结果
开始执行
开始运行
第2次结果
开始执行
开始运行
第3次结果
开始执行
开始运行
第4次结果
开始执行
开始运行
第5次结果
开始执行
开始运行
第6次结果
开始执行
开始运行
第7次结果
开始执行
开始运行
第8次结果
开始执行
开始运行
第9次结果
开始执行

In [18]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
df=pd.read_csv("G:\\data_all\\askubuntu_c2a\\data_1.csv")
ego=df["ego"]
alter=df["alter"]
ego_list=list((set(list(ego))).union(set(list(alter))))
print(len(ego_list))

4538


In [22]:
import pandas as pd

df_users = pd.read_csv("G:\\data_all\\askubuntu_c2a\\filtered_results.csv")  # Replace with the path of the first file
users = df_users['ego']

df_data =  pd.read_excel("G:\\data_all\\askubuntu_c2a\\M\\02_MM.xlsx") # Replace with the path of the second file

result = []
for i, row in enumerate(df_data.itertuples(index=False)):
    for j, value in enumerate(row):
        result.append([users[i], j, value])

result_df = pd.DataFrame(result, columns=['ego', 'M_index', 'weight'])

result_df.to_excel('G:\\data_all\\askubuntu_c2a\\M\\M_data_3.xlsx', index=False)